In [1]:
import sys
import os
import gc
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 存储数据的根目录
ROOT_PATH = "./data"
# 比赛数据集路径
DATASET_PATH = os.path.join(ROOT_PATH, "wechat_algo_data1")
# 训练集
USER_ACTION = os.path.join(DATASET_PATH, "user_action.csv")
FEED_INFO = os.path.join(DATASET_PATH, "feed_info.csv")
FEED_EMBEDDINGS = os.path.join(DATASET_PATH, "feed_embeddings.csv")
# 测试集
TEST_FILE = os.path.join(DATASET_PATH, "test_a.csv")

END_DAY = 15
SEED = 2021
pd.set_option('display.max_columns', None)

In [2]:
user_action = pd.read_csv(USER_ACTION, sep=",")
feed_info = pd.read_csv(FEED_INFO, sep=",")
test = pd.read_csv(TEST_FILE)
test["date_"] = 15
user_action = pd.concat([user_action, test])
data = pd.merge(user_action, feed_info, on="feedid", how="left")
del user_action, feed_info, test
gc.collect()
data['index'] = data['date_'].apply(lambda x: 'TRAIN' if x < 14 else('VALIDATE' if x == 14 else 'TEST'))
data.head()

,userid,feedid,date_,device,read_comment,comment,like,play,stay,click_avatar,forward,follow,favorite,authorid,videoplayseconds,description,ocr,asr,bgm_song_id,bgm_singer_id,manual_keyword_list,machine_keyword_list,manual_tag_list,machine_tag_list,description_char,ocr_char,asr_char,index
0,8,71474,1,1,0.0,0.0,1.0,500.0,5366.0,0.0,0.0,0.0,0.0,1528,11,110142 37527 123078 110142 134872 294 146821 2...,50275 5643 42751 120238 83167 1371 93533 89031...,104002 104002 104002 104002 104002 104002 1040...,13745.0,3556.0,NaN,7978;9680,219;6;124,266 0.68073416;41 0.68073416,24513 8512 27464 24513 30027 73 23103 6243 287...,11458 1254 28079 20753 23952 10212 8740 32214 ...,23112 23112 23112 23112 23112 23112 23112 2311...,TRAIN
1,8,73916,1,1,0.0,0.0,0.0,250.0,1533.0,0.0,0.0,0.0,0.0,1442,16,NaN,NaN,117252 44399 22662 4438,NaN,NaN,NaN,4388;24079;19399;19166;22371;5615,308;6,266 0.39245349;261 0.39245349;8 3.8e-07;306 0....,NaN,NaN,26018 10043 30420 969,TRAIN
2,8,50282,1,1,0.0,0.0,0.0,750.0,1302.0,0.0,0.0,0.0,0.0,8648,31,95647 49085 1824 140316 56344 45723 121036 977...,NaN,104002 104002 104002 104002 104002 104002 1040...,NaN,NaN,NaN,5077;17881;5132;25008;61;17812,9;6;222;222,16 0.33405718;332 0.33405718;8 7e-08;306 0.0;2...,21277 21277 1873 3110 32556 22468 20201 12785 ...,NaN,23112 23112 23112 23112 23112 23112 23112 2311...,TRAIN
3,8,11391,1,1,0.0,0.0,1.0,3750.0,5191.0,0.0,0.0,0.0,0.0,11976,6,32715 49575 54606 85056 90843 23031 61533 9356...,27737 129801 37630 755 95811 95811,142955 80924 45012 25794 16650 8097,13096.0,5012.0,24390;14546;6355,17918;131;22816,228;12;159;6,12 0.867993;228 0.867993;8 6.3e-07;306 0.0;207...,7357 22439 31380 12371 18881 26018 2223 10045 ...,6244 28921 12837 199 30086,2203 26439 17681 17681 10226 8481 3703 1872,TRAIN
4,8,27349,1,1,0.0,0.0,0.0,250.0,800.0,0.0,0.0,0.0,0.0,4370,12,117910 44854 26564 51353 117763 57127 134820 1...,3195 65625 79 89307 93563 62197 127203 25794 2...,150623 17721 287 66591 4438,22215.0,7899.0,9868;27100;11411,23257;20650;23257;8693;26486;25995;7144;19699,322;13;159;6,266 0.78305119;41 0.78305119;8 0.0;306 0.0;207...,17297 28718 10155 5967 17297 14126 26193 20860...,13519 14749 12 19791 20857 14040 22468 89 8481...,5990 15640 30749 29499 66 4956 7578 969,TRAIN


In [1]:
data.describe()

NameError: name 'data' is not defined

In [14]:
keyword = list(data["manual_keyword_list"]) + list(data["machine_keyword_list"])
words = []
for word in keyword:
    if str(word) == 'nan':
        continue
    else:
        words += str(word).split(';')
words = [int(w) for w in words]

tag = list(data["manual_tag_list"])
tags = []
for t in tag:
    if str(t) == 'nan':
        continue
    else:
        tags += str(t).split(';')
tags = [int(t) for t in tags]

print(words[:10])
print(tags[:10])

[24390, 14546, 6355, 9868, 27100, 11411, 6297, 20222, 18863, 7377]
[219, 6, 124, 308, 6, 9, 6, 222, 222, 228]


In [15]:
from sklearn.preprocessing import OneHotEncoder
word_encoder = OneHotEncoder(sparse=False)
word_encoder.fit(np.array(words).reshape(-1, 1))
tag_encoder = OneHotEncoder(sparse=False)
tag_encoder.fit(np.array(tags).reshape(-1, 1))

OneHotEncoder(sparse=False)

In [ ]:
def word_encode(x):
    words = []
    if str(x) == 'nan':
        words = []
    else:
        words = str(x).split(";")
    words = [int(w) for w in words]
    a = word_encoder.transform(np.array(words).reshape(-1, 1))
    return np.sum(a, axis=0)/len(a)

data["machine_keyword"] = data["machine_keyword_list"].apply(lambda x: x.split(";"))

In [28]:
a = word_encoder.transform(np.array([14546, 9868, 11411]).reshape(-1, 1))
a = np.sum(a, axis=0)/len(a)
print(len(a))
b = tag_encoder.transform(np.array([219, 124, 6]).reshape(-1, 1))
b = np.sum(b, axis=0)/len(b)

26165


In [12]:
print("read_comment ratio:", len(data[data["read_comment"]==1])/len(data))
print("like ratio:", len(data[data["like"]==1])/len(data))
print("click_avatar ratio:", len(data[data["click_avatar"]==1])/len(data))
print("forward ratio:", len(data[data["forward"]==1])/len(data))

read_comment ratio: 0.033106770439337005
like ratio: 0.024397964461146424
click_avatar ratio: 0.007122603011137013
forward ratio: 0.003612852778994781


In [13]:
subsample = {'read_comment':0.2, 'like':0.2, 'click_avatar':0.1, 'forward':0.1}

In [3]:
def reduce_mem(df, cols):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in tqdm(cols):
        col_type = df[col].dtypes
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('{:.2f} Mb, {:.2f} Mb ({:.2f} %)'.format(start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    gc.collect()
    return df

In [4]:
uf_features = ["userid", "feedid", "date_", "device", "read_comment", "like", "click_avatar",  "forward", "comment", "follow", "favorite", "play", "stay", "index"]
feed_features = ["authorid", "videoplayseconds", "bgm_song_id", "bgm_singer_id"]
features = uf_features + feed_features
df = data[features]
df["videoplayseconds"] *= 1000
df["is_finish"] = df["play"]/df["videoplayseconds"] >= 0.9
df["play_times"] = df["play"]/df["videoplayseconds"]
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,userid,feedid,date_,device,read_comment,like,click_avatar,forward,comment,follow,favorite,play,stay,index,authorid,videoplayseconds,bgm_song_id,bgm_singer_id,is_finish,play_times
0,8,71474,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,500.0,5366.0,TRAIN,1528,11000,13745.0,3556.0,False,0.045455
1,8,73916,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,1533.0,TRAIN,1442,16000,NaN,NaN,False,0.015625
2,8,50282,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,750.0,1302.0,TRAIN,8648,31000,NaN,NaN,False,0.024194
3,8,11391,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3750.0,5191.0,TRAIN,11976,6000,13096.0,5012.0,False,0.625000
4,8,27349,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,800.0,TRAIN,4370,12000,22215.0,7899.0,False,0.020833


In [5]:
df.describe()

,userid,feedid,date_,device,read_comment,like,click_avatar,forward,comment,follow,favorite,play,stay,authorid,videoplayseconds,bgm_song_id,bgm_singer_id,play_times
count,7.739867e+06,7.739867e+06,7.739867e+06,7.739867e+06,7.317882e+06,7.317882e+06,7.317882e+06,7.317882e+06,7.317882e+06,7.317882e+06,7.317882e+06,7.317882e+06,7.317882e+06,7.739867e+06,7.739867e+06,3.850313e+06,3.850313e+06,7.317882e+06
mean,1.249531e+05,5.671898e+04,8.193927e+00,1.764675e+00,3.501587e-02,2.580487e-02,7.533327e-03,3.821188e-03,4.046253e-04,7.211103e-04,1.342465e-03,2.631760e+04,3.101158e+04,9.521707e+03,3.216052e+04,1.243008e+04,8.604943e+03,9.686956e-01
std,7.239047e+04,3.276328e+04,4.276158e+00,4.242021e-01,1.838199e-01,1.585528e-01,8.646720e-02,6.169754e-02,2.011123e-02,2.684381e-02,3.661506e-02,6.477679e+04,1.013239e+05,5.388485e+03,1.071740e+05,7.297984e+03,4.969968e+03,3.086150e+00
min,8.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+03,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.132600e+04,2.821500e+04,4.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.017000e+03,5.189000e+03,4.878000e+03,1.400000e+04,6.003000e+03,4.511000e+03,7.692308e-02
50%,1.256880e+05,5.685300e+04,8.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.328900e+04,1.782900e+04,9.747000e+03,2.500000e+04,1.236500e+04,8.338000e+03,7.678462e-01
75%,1.879030e+05,8.521400e+04,1.200000e+01,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.600000e+04,4.133900e+04,1.411700e+04,5.300000e+04,1.872500e+04,1.289100e+04,1.318917e+00
max,2.502360e+05,1.128710e+05,1.500000e+01,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,3.855337e+07,8.262444e+07,1.878800e+04,2.520000e+07,2.515800e+04,1.749900e+04,3.189513e+03


In [ ]:
from tqdm import tqdm
n_day = 5
maxday = 15

play_cols = ['is_finish', 'play_times', 'play', 'stay']
y_list = ['read_comment', 'like', 'click_avatar', 'forward', 'favorite', 'comment', 'follow']

for stat_cols in tqdm([['userid'], ['feedid'], ['authorid'], ['bgm_song_id'], ['bgm_singer_id'],
                       ['userid', 'authorid'], ['userid', 'bgm_song_id'], ['userid', 'bgm_singer_id']]):
    for target_day in range(2, maxday+1):
        f = '_'.join(stat_cols)
        stat_df = pd.DataFrame()
        left, right = max(target_day - n_day, 1), target_day-1
        tmp = df[((df['date_'] >= left) & (df['date_'] <= right))].reset_index(drop=True)
        tmp['date_'] = target_day
        tmp['{}_{}day_count'.format(f, n_day)] = tmp.groupby(stat_cols)['date_'].transform('count')
        g = tmp.groupby(stat_cols)
        tmp['{}_{}day_finish_rate'.format(f, n_day)] = g[play_cols[0]].transform('mean')
        feats = ['{}_{}day_count'.format(f, n_day), '{}_{}day_finish_rate'.format(f, n_day)]

        for x in play_cols[1:]:
            for stat in ['mean', 'max']:
                tmp['{}_{}day_{}_{}'.format(f, n_day, x, stat)] = g[x].transform(stat)
                feats.append('{}_{}day_{}_{}'.format(f, n_day, x, stat))

        for y in y_list:
            tmp['{}_{}day_{}_sum'.format(f, n_day, y)] = g[y].transform('sum')
            tmp['{}_{}day_{}_mean'.format(f, n_day, y)] = g[y].transform('mean')
            feats.extend(['{}_{}day_{}_sum'.format(f, n_day, y), '{}_{}day_{}_mean'.format(f, n_day, y)])

        tmp = tmp[stat_cols + feats + ['date_']].drop_duplicates(stat_cols + ['date_']).reset_index(drop=True)
        stat_df = pd.concat([stat_df, tmp], axis=0, ignore_index=True)
        del g, tmp
    df = df.merge(stat_df, on=stat_cols + ['date_'], how='left')
    del stat_df


 75%|███████▌  | 6/8 [05:47<02:27, 73.54s/it]

In [7]:
for f in tqdm(['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id']):
    df[f + '_count'] = df[f].map(df[f].value_counts())
for f1, f2 in tqdm([['userid', 'feedid'], ['userid', 'authorid'], ['userid', 'bgm_song_id'], ['userid', 'bgm_singer_id']]):
    df['{}_in_{}_nunique'.format(f1, f2)] = df.groupby(f2)[f1].transform('nunique')
    df['{}_in_{}_nunique'.format(f2, f1)] = df.groupby(f1)[f2].transform('nunique')
for f1, f2 in tqdm([['userid', 'authorid'], ['userid', 'bgm_song_id'], ['userid', 'bgm_singer_id']]):
    df['{}_{}_count'.format(f1, f2)] = df.groupby([f1, f2])['date_'].transform('count')
    df['{}_in_{}_count_prop'.format(f1, f2)] = df['{}_{}_count'.format(f1, f2)] / (df[f2 + '_count'] + 1)
    df['{}_in_{}_count_prop'.format(f2, f1)] = df['{}_{}_count'.format(f1, f2)] / (df[f1 + '_count'] + 1)
df['videoplayseconds_in_userid_mean'] = df.groupby('userid')['videoplayseconds'].transform('mean')
df['videoplayseconds_in_authorid_mean'] = df.groupby('authorid')['videoplayseconds'].transform('mean')
df['feedid_in_authorid_nunique'] = df.groupby('authorid')['feedid'].transform('nunique')

100%|██████████| 3/3 [00:05<00:00,  1.68s/it]


In [8]:
df = df[df["date_"]>2]
df.describe()

,userid,feedid,date_,device,read_comment,like,click_avatar,forward,comment,follow,favorite,play,stay,authorid,videoplayseconds,bgm_song_id,bgm_singer_id,play_times,userid_5day_count,userid_5day_finish_rate,userid_5day_play_times_mean,userid_5day_play_times_max,userid_5day_play_mean,userid_5day_play_max,userid_5day_stay_mean,userid_5day_stay_max,userid_5day_read_comment_sum,userid_5day_read_comment_mean,userid_5day_like_sum,userid_5day_like_mean,userid_5day_click_avatar_sum,userid_5day_click_avatar_mean,userid_5day_forward_sum,userid_5day_forward_mean,userid_5day_favorite_sum,userid_5day_favorite_mean,userid_5day_comment_sum,userid_5day_comment_mean,userid_5day_follow_sum,userid_5day_follow_mean,feedid_5day_count,feedid_5day_finish_rate,feedid_5day_play_times_mean,feedid_5day_play_times_max,feedid_5day_play_mean,feedid_5day_play_max,feedid_5day_stay_mean,feedid_5day_stay_max,feedid_5day_read_comment_sum,feedid_5day_read_comment_mean,feedid_5day_like_sum,feedid_5day_like_mean,feedid_5day_click_avatar_sum,feedid_5day_click_avatar_mean,feedid_5day_forward_sum,feedid_5day_forward_mean,feedid_5day_favorite_sum,feedid_5day_favorite_mean,feedid_5day_comment_sum,feedid_5day_comment_mean,feedid_5day_follow_sum,feedid_5day_follow_mean,authorid_5day_count,authorid_5day_finish_rate,authorid_5day_play_times_mean,authorid_5day_play_times_max,authorid_5day_play_mean,authorid_5day_play_max,authorid_5day_stay_mean,authorid_5day_stay_max,authorid_5day_read_comment_sum,authorid_5day_read_comment_mean,authorid_5day_like_sum,authorid_5day_like_mean,authorid_5day_click_avatar_sum,authorid_5day_click_avatar_mean,authorid_5day_forward_sum,authorid_5day_forward_mean,authorid_5day_favorite_sum,authorid_5day_favorite_mean,authorid_5day_comment_sum,authorid_5day_comment_mean,authorid_5day_follow_sum,authorid_5day_follow_mean,bgm_song_id_5day_count,bgm_song_id_5day_finish_rate,bgm_song_id_5day_play_times_mean,bgm_song_id_5day_play_times_max,bgm_song_id_5day_play_mean,bgm_song_id_5day_play_max,bgm_song_id_5day_stay_mean,bgm_song_id_5day_stay_max,bgm_song_id_5day_read_comment_sum,bgm_song_id_5day_read_comment_mean,bgm_song_id_5day_like_sum,bgm_song_id_5day_like_mean,bgm_song_id_5day_click_avatar_sum,bgm_song_id_5day_click_avatar_mean,bgm_song_id_5day_forward_sum,bgm_song_id_5day_forward_mean,bgm_song_id_5day_favorite_sum,bgm_song_id_5day_favorite_mean,bgm_song_id_5day_comment_sum,bgm_song_id_5day_comment_mean,bgm_song_id_5day_follow_sum,bgm_song_id_5day_follow_mean,bgm_singer_id_5day_count,bgm_singer_id_5day_finish_rate,bgm_singer_id_5day_play_times_mean,bgm_singer_id_5day_play_times_max,bgm_singer_id_5day_play_mean,bgm_singer_id_5day_play_max,bgm_singer_id_5day_stay_mean,bgm_singer_id_5day_stay_max,bgm_singer_id_5day_read_comment_sum,bgm_singer_id_5day_read_comment_mean,bgm_singer_id_5day_like_sum,bgm_singer_id_5day_like_mean,bgm_singer_id_5day_click_avatar_sum,bgm_singer_id_5day_click_avatar_mean,bgm_singer_id_5day_forward_sum,bgm_singer_id_5day_forward_mean,bgm_singer_id_5day_favorite_sum,bgm_singer_id_5day_favorite_mean,bgm_singer_id_5day_comment_sum,bgm_singer_id_5day_comment_mean,bgm_singer_id_5day_follow_sum,bgm_singer_id_5day_follow_mean,userid_authorid_5day_count,userid_authorid_5day_finish_rate,userid_authorid_5day_play_times_mean,userid_authorid_5day_play_times_max,userid_authorid_5day_play_mean,userid_authorid_5day_play_max,userid_authorid_5day_stay_mean,userid_authorid_5day_stay_max,userid_authorid_5day_read_comment_sum,userid_authorid_5day_read_comment_mean,userid_authorid_5day_like_sum,userid_authorid_5day_like_mean,userid_authorid_5day_click_avatar_sum,userid_authorid_5day_click_avatar_mean,userid_authorid_5day_forward_sum,userid_authorid_5day_forward_mean,userid_authorid_5day_favorite_sum,userid_authorid_5day_favorite_mean,userid_authorid_5day_comment_sum,userid_authorid_5day_comment_mean,userid_authorid_5day_follow_sum,userid_authorid_5day_follow_mean,userid_bgm_song_id_5day_count,userid_bgm_song_id_5day_finish_rate,userid_bgm_song_id_5day_

In [9]:
from sklearn.decomposition import PCA
def feed_embedding_pca():
    feed_embedding = pd.read_csv(FEED_EMBEDDINGS)
    feed = np.array(feed_embedding.feed_embedding.str.split().tolist())
    print(feed.shape)
    pca = PCA(n_components=64)
    res = pca.fit_transform(feed)
    print(res.shape)
    df = pd.DataFrame(res)
    df['feedid'] = feed_embedding['feedid']
    return df
feed_embedding = feed_embedding_pca()
feed_embedding = feed_embedding.set_index('feedid')

(106444, 512)
(106444, 64)


In [10]:
df = pd.merge(df, feed_embedding, on="feedid", how="left")
df = reduce_mem(df, [f for f in df.columns if f not in ['date_'] + play_cols + y_list])
df.head()

100%|██████████| 273/273 [05:51<00:00,  1.29s/it]


14791.24 Mb, 4694.81 Mb (68.26 %)


,userid,feedid,date_,device,read_comment,like,click_avatar,forward,comment,follow,favorite,play,stay,index,authorid,videoplayseconds,bgm_song_id,bgm_singer_id,is_finish,play_times,userid_5day_count,userid_5day_finish_rate,userid_5day_play_times_mean,userid_5day_play_times_max,userid_5day_play_mean,userid_5day_play_max,userid_5day_stay_mean,userid_5day_stay_max,userid_5day_read_comment_sum,userid_5day_read_comment_mean,userid_5day_like_sum,userid_5day_like_mean,userid_5day_click_avatar_sum,userid_5day_click_avatar_mean,userid_5day_forward_sum,userid_5day_forward_mean,userid_5day_favorite_sum,userid_5day_favorite_mean,userid_5day_comment_sum,userid_5day_comment_mean,userid_5day_follow_sum,userid_5day_follow_mean,feedid_5day_count,feedid_5day_finish_rate,feedid_5day_play_times_mean,feedid_5day_play_times_max,feedid_5day_play_mean,feedid_5day_play_max,feedid_5day_stay_mean,feedid_5day_stay_max,feedid_5day_read_comment_sum,feedid_5day_read_comment_mean,feedid_5day_like_sum,feedid_5day_like_mean,feedid_5day_click_avatar_sum,feedid_5day_click_avatar_mean,feedid_5day_forward_sum,feedid_5day_forward_mean,feedid_5day_favorite_sum,feedid_5day_favorite_mean,feedid_5day_comment_sum,feedid_5day_comment_mean,feedid_5day_follow_sum,feedid_5day_follow_mean,authorid_5day_count,authorid_5day_finish_rate,authorid_5day_play_times_mean,authorid_5day_play_times_max,authorid_5day_play_mean,authorid_5day_play_max,authorid_5day_stay_mean,authorid_5day_stay_max,authorid_5day_read_comment_sum,authorid_5day_read_comment_mean,authorid_5day_like_sum,authorid_5day_like_mean,authorid_5day_click_avatar_sum,authorid_5day_click_avatar_mean,authorid_5day_forward_sum,authorid_5day_forward_mean,authorid_5day_favorite_sum,authorid_5day_favorite_mean,authorid_5day_comment_sum,authorid_5day_comment_mean,authorid_5day_follow_sum,authorid_5day_follow_mean,bgm_song_id_5day_count,bgm_song_id_5day_finish_rate,bgm_song_id_5day_play_times_mean,bgm_song_id_5day_play_times_max,bgm_song_id_5day_play_mean,bgm_song_id_5day_play_max,bgm_song_id_5day_stay_mean,bgm_song_id_5day_stay_max,bgm_song_id_5day_read_comment_sum,bgm_song_id_5day_read_comment_mean,bgm_song_id_5day_like_sum,bgm_song_id_5day_like_mean,bgm_song_id_5day_click_avatar_sum,bgm_song_id_5day_click_avatar_mean,bgm_song_id_5day_forward_sum,bgm_song_id_5day_forward_mean,bgm_song_id_5day_favorite_sum,bgm_song_id_5day_favorite_mean,bgm_song_id_5day_comment_sum,bgm_song_id_5day_comment_mean,bgm_song_id_5day_follow_sum,bgm_song_id_5day_follow_mean,bgm_singer_id_5day_count,bgm_singer_id_5day_finish_rate,bgm_singer_id_5day_play_times_mean,bgm_singer_id_5day_play_times_max,bgm_singer_id_5day_play_mean,bgm_singer_id_5day_play_max,bgm_singer_id_5day_stay_mean,bgm_singer_id_5day_stay_max,bgm_singer_id_5day_read_comment_sum,bgm_singer_id_5day_read_comment_mean,bgm_singer_id_5day_like_sum,bgm_singer_id_5day_like_mean,bgm_singer_id_5day_click_avatar_sum,bgm_singer_id_5day_click_avatar_mean,bgm_singer_id_5day_forward_sum,bgm_singer_id_5day_forward_mean,bgm_singer_id_5day_favorite_sum,bgm_singer_id_5day_favorite_mean,bgm_singer_id_5day_comment_sum,bgm_singer_id_5day_comment_mean,bgm_singer_id_5day_follow_sum,bgm_singer_id_5day_follow_mean,userid_authorid_5day_count,userid_authorid_5day_finish_rate,userid_authorid_5day_play_times_mean,userid_authorid_5day_play_times_max,userid_authorid_5day_play_mean,userid_authorid_5day_play_max,userid_authorid_5day_stay_mean,userid_authorid_5day_stay_max,userid_authorid_5day_read_comment_sum,userid_authorid_5day_read_comment_mean,userid_authorid_5day_like_sum,userid_authorid_5day_like_mean,userid_authorid_5day_click_avatar_sum,userid_authorid_5day_click_avatar_mean,userid_authorid_5day_forward_sum,userid_authorid_5day_forward_mean,userid_authorid_5day_favorite_sum,userid_authorid_5day_favorite_mean,userid_authorid_5day_comment_sum,userid_authorid_5day_comment_mean,userid_authorid_5day_follow_sum,userid_authorid_5day_follow_mean,userid_bgm_song_id_5day_count,userid_bgm_song_id_5day_finish_rate,userid_b

In [21]:
df.head()

,userid,feedid,date_,device,read_comment,like,click_avatar,forward,comment,follow,favorite,play,stay,authorid,videoplayseconds,bgm_song_id,bgm_singer_id,is_finish,play_times,userid_5day_count,userid_5day_finish_rate,userid_5day_play_times_mean,userid_5day_play_times_max,userid_5day_play_mean,userid_5day_play_max,userid_5day_stay_mean,userid_5day_stay_max,userid_5day_read_comment_sum,userid_5day_read_comment_mean,userid_5day_like_sum,userid_5day_like_mean,userid_5day_click_avatar_sum,userid_5day_click_avatar_mean,userid_5day_forward_sum,userid_5day_forward_mean,userid_5day_favorite_sum,userid_5day_favorite_mean,userid_5day_comment_sum,userid_5day_comment_mean,userid_5day_follow_sum,userid_5day_follow_mean,feedid_5day_count,feedid_5day_finish_rate,feedid_5day_play_times_mean,feedid_5day_play_times_max,feedid_5day_play_mean,feedid_5day_play_max,feedid_5day_stay_mean,feedid_5day_stay_max,feedid_5day_read_comment_sum,feedid_5day_read_comment_mean,feedid_5day_like_sum,feedid_5day_like_mean,feedid_5day_click_avatar_sum,feedid_5day_click_avatar_mean,feedid_5day_forward_sum,feedid_5day_forward_mean,feedid_5day_favorite_sum,feedid_5day_favorite_mean,feedid_5day_comment_sum,feedid_5day_comment_mean,feedid_5day_follow_sum,feedid_5day_follow_mean,authorid_5day_count,authorid_5day_finish_rate,authorid_5day_play_times_mean,authorid_5day_play_times_max,authorid_5day_play_mean,authorid_5day_play_max,authorid_5day_stay_mean,authorid_5day_stay_max,authorid_5day_read_comment_sum,authorid_5day_read_comment_mean,authorid_5day_like_sum,authorid_5day_like_mean,authorid_5day_click_avatar_sum,authorid_5day_click_avatar_mean,authorid_5day_forward_sum,authorid_5day_forward_mean,authorid_5day_favorite_sum,authorid_5day_favorite_mean,authorid_5day_comment_sum,authorid_5day_comment_mean,authorid_5day_follow_sum,authorid_5day_follow_mean,bgm_song_id_5day_count,bgm_song_id_5day_finish_rate,bgm_song_id_5day_play_times_mean,bgm_song_id_5day_play_times_max,bgm_song_id_5day_play_mean,bgm_song_id_5day_play_max,bgm_song_id_5day_stay_mean,bgm_song_id_5day_stay_max,bgm_song_id_5day_read_comment_sum,bgm_song_id_5day_read_comment_mean,bgm_song_id_5day_like_sum,bgm_song_id_5day_like_mean,bgm_song_id_5day_click_avatar_sum,bgm_song_id_5day_click_avatar_mean,bgm_song_id_5day_forward_sum,bgm_song_id_5day_forward_mean,bgm_song_id_5day_favorite_sum,bgm_song_id_5day_favorite_mean,bgm_song_id_5day_comment_sum,bgm_song_id_5day_comment_mean,bgm_song_id_5day_follow_sum,bgm_song_id_5day_follow_mean,bgm_singer_id_5day_count,bgm_singer_id_5day_finish_rate,bgm_singer_id_5day_play_times_mean,bgm_singer_id_5day_play_times_max,bgm_singer_id_5day_play_mean,bgm_singer_id_5day_play_max,bgm_singer_id_5day_stay_mean,bgm_singer_id_5day_stay_max,bgm_singer_id_5day_read_comment_sum,bgm_singer_id_5day_read_comment_mean,bgm_singer_id_5day_like_sum,bgm_singer_id_5day_like_mean,bgm_singer_id_5day_click_avatar_sum,bgm_singer_id_5day_click_avatar_mean,bgm_singer_id_5day_forward_sum,bgm_singer_id_5day_forward_mean,bgm_singer_id_5day_favorite_sum,bgm_singer_id_5day_favorite_mean,bgm_singer_id_5day_comment_sum,bgm_singer_id_5day_comment_mean,bgm_singer_id_5day_follow_sum,bgm_singer_id_5day_follow_mean,userid_authorid_5day_count,userid_authorid_5day_finish_rate,userid_authorid_5day_play_times_mean,userid_authorid_5day_play_times_max,userid_authorid_5day_play_mean,userid_authorid_5day_play_max,userid_authorid_5day_stay_mean,userid_authorid_5day_stay_max,userid_authorid_5day_read_comment_sum,userid_authorid_5day_read_comment_mean,userid_authorid_5day_like_sum,userid_authorid_5day_like_mean,userid_authorid_5day_click_avatar_sum,userid_authorid_5day_click_avatar_mean,userid_authorid_5day_forward_sum,userid_authorid_5day_forward_mean,userid_authorid_5day_favorite_sum,userid_authorid_5day_favorite_mean,userid_authorid_5day_comment_sum,userid_authorid_5day_comment_mean,userid_authorid_5day_follow_sum,userid_authorid_5day_follow_mean,userid_bgm_song_id_5day_count,userid_bgm_song_id_5day_finish_rate,userid_bgm_son

In [ ]:
from collections import defaultdict
def uAUC(labels, preds, user_id_list):
    """Calculate user AUC"""
    user_pred = defaultdict(lambda: [])
    user_truth = defaultdict(lambda: [])
    for idx, truth in enumerate(labels):
        user_id = user_id_list[idx]
        pred = preds[idx]
        truth = labels[idx]
        user_pred[user_id].append(pred)
        user_truth[user_id].append(truth)

    user_flag = defaultdict(lambda: False)
    for user_id in set(user_id_list):
        truths = user_truth[user_id]
        flag = False
        # 若全是正样本或全是负样本，则flag为False
        for i in range(len(truths) - 1):
            if truths[i] != truths[i + 1]:
                flag = True
                break
        user_flag[user_id] = flag

    total_auc = 0.0
    size = 0.0
    for user_id in user_flag:
        if user_flag[user_id]:
            auc = fast_auc(np.asarray(user_truth[user_id]), np.asarray(user_pred[user_id]))
            total_auc += auc 
            size += 1.0
    user_auc = float(total_auc)/size
    return user_auc

In [16]:
import lightgbm as lgb
from sklearn.metrics import roc_auc_score

cols = [f for f in df.columns if f not in ['date_', 'index'] + play_cols + y_list]
train = df[df["date_"] <= 14]
cv = df[df["date_"] == 14]
test = df[df["date_"] == 15]

uauc_list = []

for y in tqdm(y_list[:4]):
    X_train = train[cols]
    Y_train = train[y]
    params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'auc',
            'boost_from_average': True,
            'train_metric': True,
            'feature_fraction_seed': 1,
            'learning_rate': 0.05,
            'is_unbalance': True,  # 当训练数据是不平衡的，正负样本相差悬殊的时候，可以将这个属性设为true,此时会自动给少的样本赋予更高的权重
            'num_leaves': 256,  # 一般设为少于2^(max_depth)
            'max_depth': -1,  # 最大的树深，设为-1时表示不限制树的深度
            'min_child_samples': 15,  # 每个叶子结点最少包含的样本数量，用于正则化，避免过拟合
            'max_bin': 200,  # 设置连续特征或大量类型的离散特征的bins的数量
            'subsample': 1,  # Subsample ratio of the training instance.
            'subsample_freq': 1,  # frequence of subsample, <=0 means no enable
            'colsample_bytree': 0.5,  # Subsample ratio of columns when constructing each tree.
            'min_child_weight': 0,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
            'subsample_for_bin': 200000,  # Number of samples for constructing bin
            'min_split_gain': 0,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
            'reg_alpha': 2.99,  # L1 regularization term on weights
            'reg_lambda': 1.9,  # L2 regularization term on weights
            'nthread': 12,
            'verbose': 0,
            'force_row_wise':True
        }
    train_matrix = lgb.Dataset(X_train, label=Y_train)
    model=lgb.train(params, train_matrix, num_boost_round=250)
    test[y] = model.predict(test[cols])
    #uauc = roc_auc_score(cv[y], y_pred)
    #uauc_list.append(uauc)

#weighted_uauc = 0.4 * uauc_list[0] + 0.3 * uauc_list[1] + 0.2 * uauc_list[2] + 0.1 * uauc_list[3]
#print(uauc_list)
#print(weighted_uauc)
columns = ['userid', 'feedid', 'read_comment', 'like', 'click_avatar', 'forward']
res = test[columns]
res.head()
res.to_csv("res.csv", index=False)

  0%|          | 0/4 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 4/4 [22:34<00:00, 338.51s/it]


In [ ]:
columns = ['userid', 'feedid', 'read_comment', 'like', 'click_avatar', 'forward']
res = test[columns]
res.head()

In [ ]:
df.to_csv("data.csv", index=False)

In [ ]:
df.head()